
# Inertia as a Form of Model Compression in Convolutional Neural Networks


> *In submission for Deep Learning Project, Spring 2025*

**Labiba Shahab**

**Ahmed Wali**


> *What you usually achieve with a standard convolution, you can accomplish efficiently using a smaller convolution + peripheral inertia mechanism (inertial filter)*
~ Group 39


## In a Gist:

Standard dxd convolution layers involve d^2 learnable parameters and d^2 computations per convolution operation. In this project, we propose an inertial convolution mechanism that dynamically decides whether a detailed convolution is necessary. The goal is to reduce both computations and learnable parameters while maintaining performance on vision tasks like MNIST classification.

Instead of learning a full dxd kernel, we use a (d-k)x(d-k) core filter to convolve a central patch. The surrounding d^2-(d-k)^2 pixels act as an inertial periphery—evaluating local divergence or “friction.” If the divergence is high, we re-apply the core filter across the full dxd region in another convolution and stack the outputs. If low, we skip detailed computation.

In the best case, we perform just d-k computation with d-k learnable parameters.
In the worst case, we perform up to d computations, but still only learn d-k parameters, hence effectively pruning the model with estimation compression.

## Background

There has been extensive research in reducing neural network complexity:

Dynamic convolutions and skip-convolutions conditionally skip expensive operations. This project is inspired from [Dynamic Sparse Convolutions](https://arxiv.org/pdf/2102.04906), [Skip Convolutions](https://openaccess.thecvf.com/content/CVPR2021/papers/Habibian_Skip-Convolutions_for_Efficient_Video_Processing_CVPR_2021_paper.pdf), and [Fractional Skipping](https://arxiv.org/abs/2001.00705)

Pruning, quantization, and knowledge distillation reduce parameters, memory, or model depth.

Our approach is inspired by these ideas but focuses on parameter reuse and friction-aware skipping, offering a novel trade-off between learning capacity and computational efficiency.

#### Novelty

Inspired from computational optimization, our project proposes a similar mechanism to do model compression by estimation.

## Baseline Models

#### MNIST 
The baseline LeNet-5 architecture used in our experiments is a direct adaptation of the official PyTorch MNIST example. We preserved all hyperparameters, architecture layers, dropout rates, optimizer, scheduler, and dataset preprocessing to ensure fair and accurate benchmarking against our proposed compressed models.

Reproducibility + Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np

Seeds and CuDNN configs for deterministic results on a GPU

In [2]:
seed = 1
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load MNIST Dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.8MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.5MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



Evaluation Function

In [4]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = correct / len(test_loader.dataset)
    print(f"Accuracy: {accuracy:.10f}")
    return accuracy

LeNet 3x3 Model
* Adapted from [PyTorch MNIST Example](https://github.com/pytorch/examples/blob/main/mnist/main.py) to use 3x3 kernels, produced as an example in the PyTorch example series on GitHub.

In [5]:
#@adapted from https://github.com/pytorch/examples/blob/main/mnist/main.py 
class LeNet3x3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

LeNet 1x1 Model
* Adapted from the above example implementation to use 1x1 kernels instead of the original 3x3 kernels. This model is used to compare the performance of the 1x1 kernel with the original 3x3 kernel in the LeNet architecture, and with the proposed inertial filter model in future.

In [6]:
class LeNet1x1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 1, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(12544, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Training Function

In [7]:
def train_model(model, name):
    model = model.to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

    for epoch in range(1, 15):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
        scheduler.step()

    print(f"\n{name} Results:")
    acc = test(model, device, test_loader)
    return acc

So Let's Evaluate MNIST on the Baseline

In [11]:
model_3x3 = LeNet3x3()
acc_3x3 = train_model(model_3x3, "LeNet 3x3")
model_1x1 = LeNet1x1()
acc_1x1 = train_model(model_1x1, "LeNet 1x1")


LeNet 3x3 Results:
Accuracy: 0.9917000000

LeNet 1x1 Results:
Accuracy: 0.9574000000


#### Note:

This dataset serves as a good starting point due to its simplicity and low variance. We will use this dataset as the initial starting point for our analysis. We will adapt the standard pytorch LeNet-5 implementation with 2 convolutional layers in 3x3 filters. We will then train a similar model with the only alteration made such that the two convolutional layers use 1x1 core filters + 1 pixel periphery, and we would compare the results. If the results are comparable, we would have effectively reduced the number of learnable parameters and potentially the computational bulk due to the smaller kernel size.

**What do we expect?**

Since the MNIST dataset is a grascale dataset, we anticipate low divergence or friction across patches in images, hence we anticipate improved performance due to the use of inertial filters.